In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from IPython.display import Image, display # For displaying the graph
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [8]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [19]:
class blogState(TypedDict):
    topic : str
    outline : str
    blog_content : str
    blog_score : str

In [20]:
def outline_gen(state: blogState) -> blogState:
    topic = state["topic"]
    template = "Generate an outline for a blog on the topic: {topic}"
    prompt = PromptTemplate.from_template(template)
    state["outline"] = llm.invoke(prompt.format(topic=topic)).content
    return state

In [21]:
def blog_gen(state: blogState) -> blogState:
    topic = state["topic"]
    outline = state["outline"]
    template = "Generate a blog for the given topic : {topic} and \n\n Outline : {outline}"
    prompt = PromptTemplate.from_template(template)
    state["blog_content"] = llm.invoke(prompt.format(topic=topic,outline=outline)).content
    return state

In [25]:
def blog_scorer(state: blogState) -> blogState:
    blog_content = state["blog_content"]
    template = "Score the following blog content on a scale of 1 to 10 based on its quality and relevance: {blog_content}"
    prompt = PromptTemplate.from_template(template)
    score_str = llm.invoke(prompt.format(blog_content=blog_content)).content
    state["blog_score"] = score_str.strip()
    return state

In [26]:
graph = StateGraph(blogState)

graph.add_node("create_outline", outline_gen)
graph.add_node("create_blog", blog_gen)
graph.add_node("score_blog", blog_scorer)

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", "score_blog")
graph.add_edge("score_blog", END)

workflow = graph.compile()

In [27]:
initial_state = {"topic": "Will AI be taking your Jobs in the coming future?"}
final_state = workflow.invoke(initial_state)

In [28]:
print(final_state["blog_content"])

## Will AI Be Taking Your Jobs? Separating Hype from Reality in the Future of Work

The headlines scream. "AI will take your job!" "Robots are coming for your livelihood!" From factory floors to creative studios, a pervasive fear has taken root: the fear that artificial intelligence, with its ever-growing capabilities, is poised to render human workers obsolete. With the rapid advancements in generative AI like ChatGPT and DALL-E, self-driving vehicles, and sophisticated automation, it's easy to feel a sense of impending doom.

But is the future of work truly a simple "yes" or "no" to AI job replacement? The reality, as with most complex technological shifts, is far more nuanced. This post will explore the complexities, arguing that the future involves profound transformation, augmentation, and the creation of new opportunities, alongside the inevitable displacement of certain tasks and jobs. It's about understanding how we can thrive in a world where humans and AI work side-by-side.



In [29]:
print(final_state["blog_score"])

## Score: 9/10

Here's a breakdown of the scoring:

**Quality (4.5/5):**
*   **Clarity and Readability:** Excellent. The language is accessible, clear, and avoids excessive jargon while still explaining complex concepts.
*   **Structure and Flow:** Very well-structured. The introduction sets the stage effectively, the headings guide the reader logically through the arguments, and the conclusion provides a strong summary and call to action.
*   **Depth and Nuance:** Outstanding. The post delivers on its promise to "separate hype from reality," offering a balanced and nuanced perspective. It clearly distinguishes between job replacement and task automation, augmentation, and the creation of new roles.
*   **Engagement:** The tone is engaging, empathetic to common fears, yet firmly informative and optimistic about human adaptability.
*   **Accuracy:** The information presented aligns with current expert consensus on AI's impact on the job market. The examples given are relevant and illust